In [ ]:
import os
import sys
import time
import random
import string

import torch
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data
import numpy as np
import torch.nn.functional as F
from nltk.metrics.distance import edit_distance

from utils import CTCLabelConverter, CTCLabelConverterForBaiduWarpctc, AttnLabelConverter, Averager
from c_dataset import custom_Batch_Balanced_Dataset,custom_dataset,AlignCollate
from model import Model
from test import validation
import easydict
global opt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

global opt
opt = easydict.EasyDict({
    "exp_name": "test_01",
    "train_data": "/data/data/STARN/data_lmdb_release/training",
    "valid_data":"/data/data/STARN/data_lmdb_release/validation",
    "manualSeed": 1111,
    "workers": 8,
    "batch_size":1024,
    "num_iter":300000,
    "valInterval":1,
    "saved_model":'',
    "FT":False,
    "adam":False,
    "lr":1,
    "beta1":0.9,
    "rho":0.95,
    "eps":1e-8,
    "grad_clip":5,
    "baiduCTC":False,
    "select_data":'ST',
    "batch_ratio":'1',
    "total_data_usage_ratio":'1.0',
    "batch_max_length":25,
    "imgW":100,
    "imgH":32,
    "rgb":False,
    "character":"0123456789abcdefghijklmnopqrstuvwxyz",
    "sensitive":False,
    "PAD":False,
    "data_filtering_off":False,
    "Transformation":"TPS",
    "FeatureExtraction":"ResNet",
    "SequenceModeling":"BiLSTM",
    "Prediction":'Attn',
    "num_fiducial":20,
    "input_channel":1,
    "output_channel":512,
    "hidden_size":256    
})





def validation(model, criterion, evaluation_loader, converter, opt):
    """ validation or evaluation """
    n_correct = 0
    norm_ED = 0
    length_of_data = 0
    infer_time = 0
    valid_loss_avg = Averager()

    for i, (image_tensors, labels) in enumerate(evaluation_loader):
        batch_size = image_tensors.size(0)
        length_of_data = length_of_data + batch_size
        image = image_tensors.to(device)
        # For max length prediction
        length_for_pred = torch.IntTensor([opt.batch_max_length] * batch_size).to(device)
        text_for_pred = torch.LongTensor(batch_size, opt.batch_max_length + 1).fill_(0).to(device)

        text_for_loss, length_for_loss = converter.encode(labels, batch_max_length=opt.batch_max_length)

        start_time = time.time()
        if 'CTC' in opt.Prediction:
            preds = model(image, text_for_pred)
            forward_time = time.time() - start_time

            # Calculate evaluation loss for CTC deocder.
            preds_size = torch.IntTensor([preds.size(1)] * batch_size)
            # permute 'preds' to use CTCloss format
            if opt.baiduCTC:
                cost = criterion(preds.permute(1, 0, 2), text_for_loss, preds_size, length_for_loss) / batch_size
            else:
                cost = criterion(preds.log_softmax(2).permute(1, 0, 2), text_for_loss, preds_size, length_for_loss)

            # Select max probabilty (greedy decoding) then decode index to character
            if opt.baiduCTC:
                _, preds_index = preds.max(2)
                preds_index = preds_index.view(-1)
            else:
                _, preds_index = preds.max(2)
            preds_str = converter.decode(preds_index.data, preds_size.data)
        
        else:
            preds = model(image, text_for_pred, is_train=False)
            forward_time = time.time() - start_time

            preds = preds[:, :text_for_loss.shape[1] - 1, :]
            target = text_for_loss[:, 1:]  # without [GO] Symbol
            cost = criterion(preds.contiguous().view(-1, preds.shape[-1]), target.contiguous().view(-1))

            # select max probabilty (greedy decoding) then decode index to character
            _, preds_index = preds.max(2)
            preds_str = converter.decode(preds_index, length_for_pred)
            labels = converter.decode(text_for_loss[:, 1:], length_for_loss)

        infer_time += forward_time
        valid_loss_avg.add(cost)

        # calculate accuracy & confidence score
        preds_prob = F.softmax(preds, dim=2)
        preds_max_prob, _ = preds_prob.max(dim=2)
        confidence_score_list = []
        for gt, pred, pred_max_prob in zip(labels, preds_str, preds_max_prob):
            if 'Attn' in opt.Prediction:
                gt = gt[:gt.find('[s]')]
                pred_EOS = pred.find('[s]')
                pred = pred[:pred_EOS]  # prune after "end of sentence" token ([s])
                pred_max_prob = pred_max_prob[:pred_EOS]


            if pred == gt:
                n_correct += 1

            '''
            (old version) ICDAR2017 DOST Normalized Edit Distance https://rrc.cvc.uab.es/?ch=7&com=tasks
            "For each word we calculate the normalized edit distance to the length of the ground truth transcription."
            if len(gt) == 0:
                norm_ED += 1
            else:
                norm_ED += edit_distance(pred, gt) / len(gt)
            '''

            # ICDAR2019 Normalized Edit Distance
            if len(gt) == 0 or len(pred) == 0:
                norm_ED += 0
            elif len(gt) > len(pred):
                norm_ED += 1 - edit_distance(pred, gt) / len(gt)
            else:
                norm_ED += 1 - edit_distance(pred, gt) / len(pred)

            # calculate confidence score (= multiply of pred_max_prob)
            try:
                confidence_score = pred_max_prob.cumprod(dim=0)[-1]
            except:
                confidence_score = 0  # for empty pred case, when prune after "end of sentence" token ([s])
            confidence_score_list.append(confidence_score)
            # print(pred, gt, pred==gt, confidence_score)

    accuracy = n_correct / float(length_of_data) * 100
    norm_ED = norm_ED / float(length_of_data)  # ICDAR2019 Normalized Edit Distance

    return valid_loss_avg.val(), accuracy, norm_ED, preds_str, confidence_score_list, labels, infer_time, length_of_data


if __name__ == '__main__':

    """ Seed and GPU setting """
    # print("Random Seed: ", opt.manualSeed)
    random.seed(opt.manualSeed)
    np.random.seed(opt.manualSeed)
    torch.manual_seed(opt.manualSeed)
    torch.cuda.manual_seed(opt.manualSeed)

    cudnn.benchmark = True
    cudnn.deterministic = True
    opt.num_gpu = torch.cuda.device_count()
    # print('device count', opt.num_gpu)
    if opt.num_gpu > 1:
        print('------ Use multi-GPU setting ------')
        print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
        # check multi-GPU issue https://github.com/clovaai/deep-text-recognition-benchmark/issues/1
        opt.workers = opt.workers * opt.num_gpu
        opt.batch_size = opt.batch_size * opt.num_gpu









    numclass_path = "/data/work_dir/img/generate_text_ko/mrjaehong_text_generation/generate_img/ch_range.txt"
    f = open(numclass_path, 'r')
    ch_temp = f.read()
    f.close()
    opt.character = ch_temp

    converter = AttnLabelConverter(opt.character)
    opt.num_class = len(converter.character)



    train_dataset = custom_dataset("/data/work_dir/img/generate_text_ko/mrjaehong_text_generation/generate_img/label.csv")
    valid_dataset = custom_dataset("/data/work_dir/img/generate_text_ko/mrjaehong_text_generation/generate_img_val/label.csv")

    AlignCollate_valid = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)

    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=opt.batch_size,
            shuffle=True,  # 'True' to check training progress with validation function.
            num_workers=int(opt.workers),
            collate_fn=AlignCollate_valid, pin_memory=True)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=opt.batch_size,
            shuffle=True,  # 'True' to check training progress with validation function.
            num_workers=int(opt.workers),
            collate_fn=AlignCollate_valid, pin_memory=True)


    if opt.rgb:
        opt.input_channel = 3
    model = Model(opt)


    for name, param in model.named_parameters():
        if 'localization_fc2' in name:
            print(f'Skip {name} as it is already initialized')
            continue
        try:
            if 'bias' in name:
                init.constant_(param, 0.0)
            elif 'weight' in name:
                init.kaiming_normal_(param)
        except Exception as e:  # for batchnorm.
            if 'weight' in name:
                param.data.fill_(1)
            continue


    model = torch.nn.DataParallel(model).to(device)



    criterion = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)

    loss_avg = Averager()


    filtered_parameters = []

    for p in filter(lambda p: p.requires_grad, model.parameters()):
        filtered_parameters.append(p)


    if opt.adam:
#         optimizer = optim.Adam(filtered_parameters, lr=opt.lr, betas=(opt.beta1, 0.999))
        optimizer = optim.Adam(filtered_parameters, lr=opt.lr)
    else:
        optimizer = optim.Adadelta(filtered_parameters, lr=opt.lr, rho=opt.rho, eps=opt.eps)

    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
    
    nb_epochs = 100000

    for epoch in range(nb_epochs + 1):
        
        for batch_idx, samples in enumerate(train_loader):

            log = open(f'./log_val.txt', 'a')
            log2= open(f'./log_train.txt', 'a')

            start_time = time.time()        
            model.train()

            image_tensors, labels = samples
            image = image_tensors.to(device)
            text, length = converter.encode(labels, batch_max_length=opt.batch_max_length)




            preds = model(image, text[:, :-1])  # align with Attention.forward
            target = text[:, 1:]  # without [GO] Symbol
            cost = criterion(preds.view(-1, preds.shape[-1]), target.contiguous().view(-1))



            model.zero_grad()
            cost.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), opt.grad_clip)  # gradient clipping with 5 (Default)
            optimizer.step()

            loss_avg.add(cost)

            for param_group in optimizer.param_groups:
                learning_rate_val=param_group['lr']


            ## 평가
            model.eval()
            with torch.no_grad():
                valid_loss, current_accuracy, current_norm_ED, preds, confidence_score, labels, infer_time, length_of_data = validation(
                            model, criterion, valid_loader, converter, opt)

            end = time.time()
            loss_log = f'epoch : {epoch} [{batch_idx}/{len(train_loader)}] Train loss: {loss_avg.val():0.5f},Valid loss: {valid_loss:0.5f}, time : {end-start_time} lr : {learning_rate_val}'        
            loss_avg.reset()


            print(loss_log)

            dashed_line = '-' * 80
            head = f'{"Ground Truth":25s} | {"Prediction":25s} | Confidence Score & T/F'
            predicted_result_log = f'{dashed_line}\n{head}\n{dashed_line}\n'
            for gt, pred, confidence in zip(labels[:5], preds[:5], confidence_score[:5]):
                if 'Attn' in opt.Prediction:
                    gt = gt[:gt.find('[s]')]
                    pred = pred[:pred.find('[s]')]

                predicted_result_log += f'{gt:25s} | {pred:25s} | {confidence:0.4f}\t{str(pred == gt)}\n'
            predicted_result_log += f'{dashed_line}'
    #         print(predicted_result_log)
            
            log2.write(loss_log + '\n')
            log.write(loss_log + '\n')
            log.write(predicted_result_log + '\n')
            log.close()
            log2.close()
            
        scheduler.step()

------ Use multi-GPU setting ------
if you stuck too long time with multi-GPU setting, try to set --workers 0
Skip Transformation.LocalizationNetwork.localization_fc2.weight as it is already initialized
Skip Transformation.LocalizationNetwork.localization_fc2.bias as it is already initialized
epoch : 0 [0/23] Train loss: 7.82387,Valid loss: 7.82160, time : 27.48381519317627 lr : 1
epoch : 0 [1/23] Train loss: 7.68975,Valid loss: 7.77373, time : 7.67116117477417 lr : 1
epoch : 0 [2/23] Train loss: 7.53181,Valid loss: 7.67954, time : 7.454776763916016 lr : 1
epoch : 0 [3/23] Train loss: 7.32899,Valid loss: 7.53975, time : 7.568933486938477 lr : 1
epoch : 0 [4/23] Train loss: 7.08693,Valid loss: 7.26721, time : 7.549732446670532 lr : 1
epoch : 0 [5/23] Train loss: 6.80533,Valid loss: 6.87814, time : 7.360023260116577 lr : 1
epoch : 0 [6/23] Train loss: 6.52310,Valid loss: 6.60615, time : 7.6156158447265625 lr : 1
epoch : 0 [7/23] Train loss: 6.34041,Valid loss: 6.40934, time : 7.676161766

epoch : 3 [14/23] Train loss: 4.78862,Valid loss: 5.27709, time : 9.585374116897583 lr : 0.9702989999999999
epoch : 3 [15/23] Train loss: 4.79550,Valid loss: 5.09032, time : 9.365378856658936 lr : 0.9702989999999999
epoch : 3 [16/23] Train loss: 4.80214,Valid loss: 5.28174, time : 9.28157114982605 lr : 0.9702989999999999
epoch : 3 [17/23] Train loss: 4.78857,Valid loss: 5.05898, time : 9.439975500106812 lr : 0.9702989999999999
epoch : 3 [18/23] Train loss: 4.80499,Valid loss: 5.29412, time : 9.047627210617065 lr : 0.9702989999999999
epoch : 3 [19/23] Train loss: 4.79327,Valid loss: 5.05423, time : 9.649041414260864 lr : 0.9702989999999999
epoch : 3 [20/23] Train loss: 4.77644,Valid loss: 5.29620, time : 9.158173561096191 lr : 0.9702989999999999
epoch : 3 [21/23] Train loss: 4.78347,Valid loss: 5.04830, time : 9.195147275924683 lr : 0.9702989999999999
epoch : 3 [22/23] Train loss: 4.78916,Valid loss: 5.27585, time : 8.774902820587158 lr : 0.9702989999999999
epoch : 4 [0/23] Train loss: 

epoch : 7 [0/23] Train loss: 4.73935,Valid loss: 5.16746, time : 9.60843801498413 lr : 0.9320653479069899
epoch : 7 [1/23] Train loss: 4.72964,Valid loss: 4.96637, time : 9.53887677192688 lr : 0.9320653479069899
epoch : 7 [2/23] Train loss: 4.75169,Valid loss: 5.16153, time : 9.503590106964111 lr : 0.9320653479069899
epoch : 7 [3/23] Train loss: 4.72844,Valid loss: 4.98539, time : 9.889378547668457 lr : 0.9320653479069899
epoch : 7 [4/23] Train loss: 4.72236,Valid loss: 5.23827, time : 9.450141429901123 lr : 0.9320653479069899
epoch : 7 [5/23] Train loss: 4.72143,Valid loss: 4.92456, time : 9.707441091537476 lr : 0.9320653479069899
epoch : 7 [6/23] Train loss: 4.73809,Valid loss: 5.13846, time : 9.494905710220337 lr : 0.9320653479069899
epoch : 7 [7/23] Train loss: 4.71569,Valid loss: 4.96715, time : 9.607858896255493 lr : 0.9320653479069899
epoch : 7 [8/23] Train loss: 4.72471,Valid loss: 5.11072, time : 9.507297992706299 lr : 0.9320653479069899
epoch : 7 [9/23] Train loss: 4.71596,Va

epoch : 10 [8/23] Train loss: 4.68523,Valid loss: 5.07034, time : 9.398581743240356 lr : 0.9043820750088043
epoch : 10 [9/23] Train loss: 4.68524,Valid loss: 4.82094, time : 9.485077142715454 lr : 0.9043820750088043
epoch : 10 [10/23] Train loss: 4.75196,Valid loss: 4.97478, time : 9.490891456604004 lr : 0.9043820750088043
epoch : 10 [11/23] Train loss: 4.68787,Valid loss: 4.93966, time : 9.904149293899536 lr : 0.9043820750088043
epoch : 10 [12/23] Train loss: 4.67595,Valid loss: 4.94579, time : 9.342158794403076 lr : 0.9043820750088043
epoch : 10 [13/23] Train loss: 4.66441,Valid loss: 4.87004, time : 9.922587156295776 lr : 0.9043820750088043
epoch : 10 [14/23] Train loss: 4.68922,Valid loss: 5.00240, time : 9.340147495269775 lr : 0.9043820750088043
epoch : 10 [15/23] Train loss: 4.69784,Valid loss: 4.82125, time : 9.913043975830078 lr : 0.9043820750088043
epoch : 10 [16/23] Train loss: 4.72493,Valid loss: 5.07546, time : 9.58562707901001 lr : 0.9043820750088043
epoch : 10 [17/23] Tra

epoch : 13 [15/23] Train loss: 4.61675,Valid loss: 4.93284, time : 9.543246746063232 lr : 0.8775210229989678
epoch : 13 [16/23] Train loss: 4.65134,Valid loss: 4.81176, time : 9.723329067230225 lr : 0.8775210229989678
epoch : 13 [17/23] Train loss: 4.68124,Valid loss: 4.88591, time : 9.961178541183472 lr : 0.8775210229989678
epoch : 13 [18/23] Train loss: 4.62272,Valid loss: 4.87030, time : 9.901655912399292 lr : 0.8775210229989678
epoch : 13 [19/23] Train loss: 4.63609,Valid loss: 4.82992, time : 9.430723905563354 lr : 0.8775210229989678
epoch : 13 [20/23] Train loss: 4.62249,Valid loss: 4.75798, time : 9.776795625686646 lr : 0.8775210229989678
epoch : 13 [21/23] Train loss: 4.68888,Valid loss: 4.85344, time : 9.636003494262695 lr : 0.8775210229989678
epoch : 13 [22/23] Train loss: 4.61771,Valid loss: 4.75323, time : 9.226398944854736 lr : 0.8775210229989678
epoch : 14 [0/23] Train loss: 4.60679,Valid loss: 4.94306, time : 9.9016592502594 lr : 0.8687458127689781
epoch : 14 [1/23] Trai

...

epoch : 273 [12/23] Train loss: 0.27827,Valid loss: 0.65617, time : 9.857476472854614 lr : 0.0643291962372672
epoch : 273 [13/23] Train loss: 0.28411,Valid loss: 0.64459, time : 9.874717950820923 lr : 0.0643291962372672
epoch : 273 [14/23] Train loss: 0.29155,Valid loss: 0.63735, time : 10.2353196144104 lr : 0.0643291962372672
epoch : 273 [15/23] Train loss: 0.27996,Valid loss: 0.64030, time : 10.113888502120972 lr : 0.0643291962372672
epoch : 273 [16/23] Train loss: 0.27496,Valid loss: 0.74947, time : 9.681767702102661 lr : 0.0643291962372672
epoch : 273 [17/23] Train loss: 0.28211,Valid loss: 0.72593, time : 10.149143695831299 lr : 0.0643291962372672
epoch : 273 [18/23] Train loss: 0.28418,Valid loss: 0.72537, time : 10.27157187461853 lr : 0.0643291962372672
epoch : 273 [19/23] Train loss: 0.28031,Valid loss: 0.62216, time : 10.698498725891113 lr : 0.0643291962372672
epoch : 273 [20/23] Train loss: 0.28016,Valid loss: 0.63848, time : 10.472503662109375 lr : 0.0643291962372672
epoch :

epoch : 276 [17/23] Train loss: 0.28223,Valid loss: 0.61499, time : 10.061864614486694 lr : 0.06241855477982412
epoch : 276 [18/23] Train loss: 0.27953,Valid loss: 0.61086, time : 10.331919431686401 lr : 0.06241855477982412
epoch : 276 [19/23] Train loss: 0.27374,Valid loss: 0.71286, time : 9.990232467651367 lr : 0.06241855477982412
epoch : 276 [20/23] Train loss: 0.28253,Valid loss: 0.71755, time : 10.181441068649292 lr : 0.06241855477982412
epoch : 276 [21/23] Train loss: 0.28539,Valid loss: 0.67567, time : 10.359015226364136 lr : 0.06241855477982412
epoch : 276 [22/23] Train loss: 0.28095,Valid loss: 0.59999, time : 9.48560357093811 lr : 0.06241855477982412
epoch : 277 [0/23] Train loss: 0.27392,Valid loss: 0.67298, time : 10.510305404663086 lr : 0.06179436923202588
epoch : 277 [1/23] Train loss: 0.28340,Valid loss: 0.70265, time : 10.205408811569214 lr : 0.06179436923202588
epoch : 277 [2/23] Train loss: 0.28070,Valid loss: 0.66820, time : 9.904259443283081 lr : 0.06179436923202588

epoch : 279 [22/23] Train loss: 0.28502,Valid loss: 0.69330, time : 9.335654973983765 lr : 0.06056466128430856
epoch : 280 [0/23] Train loss: 0.27172,Valid loss: 0.71449, time : 10.415850639343262 lr : 0.05995901467146548
epoch : 280 [1/23] Train loss: 0.27480,Valid loss: 0.66496, time : 10.28220248222351 lr : 0.05995901467146548
epoch : 280 [2/23] Train loss: 0.27414,Valid loss: 0.71166, time : 10.33193063735962 lr : 0.05995901467146548
epoch : 280 [3/23] Train loss: 0.28132,Valid loss: 0.62289, time : 10.305082321166992 lr : 0.05995901467146548
epoch : 280 [4/23] Train loss: 0.28336,Valid loss: 0.67324, time : 10.386869192123413 lr : 0.05995901467146548
epoch : 280 [5/23] Train loss: 0.27218,Valid loss: 0.65585, time : 10.569070816040039 lr : 0.05995901467146548
epoch : 280 [6/23] Train loss: 0.26597,Valid loss: 0.61741, time : 10.607264757156372 lr : 0.05995901467146548
epoch : 280 [7/23] Train loss: 0.27941,Valid loss: 0.63052, time : 10.531813621520996 lr : 0.05995901467146548
epo

epoch : 283 [4/23] Train loss: 0.28284,Valid loss: 0.71008, time : 10.013160228729248 lr : 0.05817817197670828
epoch : 283 [5/23] Train loss: 0.27734,Valid loss: 0.67080, time : 10.561294555664062 lr : 0.05817817197670828
epoch : 283 [6/23] Train loss: 0.27674,Valid loss: 0.64749, time : 10.174068212509155 lr : 0.05817817197670828
epoch : 283 [7/23] Train loss: 0.27013,Valid loss: 0.63365, time : 9.932427406311035 lr : 0.05817817197670828
epoch : 283 [8/23] Train loss: 0.28788,Valid loss: 0.68193, time : 10.078747034072876 lr : 0.05817817197670828
epoch : 283 [9/23] Train loss: 0.27804,Valid loss: 0.67826, time : 10.39067816734314 lr : 0.05817817197670828
epoch : 283 [10/23] Train loss: 0.27186,Valid loss: 0.70812, time : 10.081612825393677 lr : 0.05817817197670828
epoch : 283 [11/23] Train loss: 0.26919,Valid loss: 0.62740, time : 10.192925214767456 lr : 0.05817817197670828
epoch : 283 [12/23] Train loss: 0.26814,Valid loss: 0.70815, time : 9.762565851211548 lr : 0.05817817197670828
e

epoch : 286 [9/23] Train loss: 0.27442,Valid loss: 0.71830, time : 10.355611085891724 lr : 0.05645022209082806
epoch : 286 [10/23] Train loss: 0.27684,Valid loss: 0.70360, time : 10.39570927619934 lr : 0.05645022209082806
epoch : 286 [11/23] Train loss: 0.27178,Valid loss: 0.73080, time : 10.574519634246826 lr : 0.05645022209082806
epoch : 286 [12/23] Train loss: 0.27833,Valid loss: 0.71959, time : 10.179269552230835 lr : 0.05645022209082806
epoch : 286 [13/23] Train loss: 0.27218,Valid loss: 0.62424, time : 10.291951417922974 lr : 0.05645022209082806
epoch : 286 [14/23] Train loss: 0.27176,Valid loss: 0.71332, time : 10.16816258430481 lr : 0.05645022209082806
epoch : 286 [15/23] Train loss: 0.26611,Valid loss: 0.72021, time : 10.230946063995361 lr : 0.05645022209082806
epoch : 286 [16/23] Train loss: 0.27372,Valid loss: 0.70495, time : 10.37497067451477 lr : 0.05645022209082806
epoch : 286 [17/23] Train loss: 0.26529,Valid loss: 0.66787, time : 10.093690156936646 lr : 0.05645022209082

In [ ]:
image_tensors, labels = train_dataset.get_batch()

In [ ]:
image_tensors.shape

In [ ]:
imzcage,dd = next(iter(valid_loader))
print(type(imzcage),type(dd))
print(imzcage.shape)

In [ ]:
https://wikidocs.net/57165
    